# 本地训练断线重连+训练描述

### 初始化SDK与Anylearn后端引擎连接

In [1]:
from anylearn.config import init_sdk

init_sdk('http://192.168.10.22:31888', 'xlearn', '123456')

### 创建快速训练任务

In [2]:
from anylearn.applications.quickstart import quick_train

train_task, algo, dset, project = quick_train(
    algorithm_dir="../../resources/cnn",
    dataset_dir="../../resources/fashion_mnist",
    entrypoint="python fashion_mnist.py",
    output="model",
    dataset_hyperparam_name="data-path",
    hyperparams={'batch-size': 256, 'epochs': 12},
    gpu_num=1,
)
train_task

/Users/yhuang/dev/anylearn-sdk/examples/venv/lib/python3.9/site-packages/sqlalchemy/sql/sqltypes.py:727: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


TrainTask(name='jiq05vb3', description='', state=0, visibility=1, creator_id='USERfb6c6d2111eaadda13fd17feeac7', owner=['USERfb6c6d2111eaadda13fd17feeac7'], project_id='PROJ6adaca8711ebab09ee8571c2da21', algorithm_id='ALGO69a0ca8711ebab09ee8571c2da21', train_params='{"data-path": "$DSETbec8c90411eb8edd8a9cbf62f009", "batch-size": 256, "epochs": 12}', files='DSETbec8c90411eb8edd8a9cbf62f009', results_id='FILEbc92ca8711ebab09ee8571c2da21', secret_key='TKEY755aca8711ebab09ee8571c2da21', create_time='2021-06-11 15:37:45', finish_time='', envs='', gpu_num=1, hpo=False, hpo_search_space=None, final_metric=None, id='TRAI2e28ca8711ebab09ee8571c2da21')

### 模拟本地进程中断

先前获取的用于管理训练任务的训练对象丢失，无法继续获取训练状态、训练结果等信息。

In [3]:
train_task = None

### 中断重连本地未完成的训练任务

通过本地快速开始（`quickstart`模块）启动训练后，训练任务本身运行在后端，与本地环境相互独立。

但在用户本地有可能遇到进程中断、宕机等情况，导致在SDK上下文中丢失已启动训练的对象，继而无法继续维护训练的生命周期。

因此SDK在快速开始模块中提供了中断重连的功能，便于用户重新获得已启动的训练的对象。

In [4]:
from anylearn.applications.quickstart import resume_unfinished_local_train_tasks

tasks = resume_unfinished_local_train_tasks()
tasks

[TrainTask(name='jiq05vb3', description='', state=0, visibility=1, creator_id='USERfb6c6d2111eaadda13fd17feeac7', owner=['USERfb6c6d2111eaadda13fd17feeac7'], project_id='PROJ6adaca8711ebab09ee8571c2da21', algorithm_id='ALGO69a0ca8711ebab09ee8571c2da21', train_params='{"data-path": "$DSETbec8c90411eb8edd8a9cbf62f009", "batch-size": 256, "epochs": 12}', files='DSETbec8c90411eb8edd8a9cbf62f009', results_id='FILEbc92ca8711ebab09ee8571c2da21', secret_key='TKEY755aca8711ebab09ee8571c2da21', create_time='2021-06-11 15:37:45', finish_time='', envs='', gpu_num=1, hpo=False, hpo_search_space=None, final_metric=None, id='TRAI2e28ca8711ebab09ee8571c2da21')]

### 获取训练任务描述

当用户运行了大量的训练后会产生大量的训练任务元信息，这些元信息来自Anylearn后端引擎，主要基于各类ID，对**用户区分和识别**每个训练任务会造成一些困难。

因此SDK中还提供了对每个本地训练任务的额外描述，主要基于用户本地的上下文信息，例如：训练启动时间、本地算法路径、数据集路径等等。

In [5]:
profiles = [t.get_train_profile() for t in tasks]
profiles

[TrainProfile(
     id='DESC5eeaca8711ebafd5acde48001122',
     train_task_id=TRAI2e28ca8711ebab09ee8571c2da21,
     entrypoint='python fashion_mnist.py',
     train_params='{"data-path": "$DSETbec8c90411eb8edd8a9cbf62f009", "batch-size": 256, "epochs": 12}',
     algorithm_id='ALGO69a0ca8711ebab09ee8571c2da21',
     algorithm_dir='../../resources/cnn',
     algorithm_archive='/Users/yhuang/.anylearn/ALGO_f0k2vp56.zip',
     dataset_id='DSETbec8c90411eb8edd8a9cbf62f009',
     dataset_dir='../../resources/fashion_mnist',
     dataset_archive='None',
     created_at=datetime.strptime('2021-06-11 15:37:38', '%Y-%m-%d %H:%M:%S'),
 )]